In [1]:
import ee
import geemap
import os
from tqdm import tqdm
import numpy as np
ee.Initialize()

In [2]:
image_collection_fire = ee.ImageCollection("ESA/CCI/FireCCI/5_1")
# changeMap_ls7 = ee.Image("users/dxj/Changemap_NE").select('yod')
# changeMap_nls7 = ee.Image("users/platuo/Changemap_ne_filterLC7_30").select('yod')
changeMap_nls7 = ee.Image("users/dxj/Changemap_ne_aft")
changeMap_ls7 = ee.Image("users/dxj/Changemap_ne_pre")
protect_area = ee.FeatureCollection("users/dxj/protect_area1")
IFZ_mask = ee.Image("users/dxj/changeMap_mask")

In [3]:
start_year = 1988
end_year = 2019

In [4]:
changeMap_ls7_0 = changeMap_ls7.unmask(0)
changeMap_nls7_0 = changeMap_nls7.unmask(0)
changeMap_mask = changeMap_ls7_0.mask(changeMap_ls7_0.And(changeMap_nls7_0).eq(1)).unmask(0)
changeMap_add = changeMap_nls7_0.add(changeMap_ls7_0)
changeMap = changeMap_add.subtract(changeMap_mask)
changeMap_0 = changeMap.eq(0).Not()
changeMap = changeMap.mask(changeMap_0)

In [5]:
# image = changeMap.select('mag')
# task = ee.batch.Export.image.toDrive(image,
# 'changeMap_mag',
# scale=30,
# maxPixels=1e13,
# # region=空间筛选边界shp.geometry()
# )
# task.start()

In [6]:
def fire_process(roi, image_collection, year):
    image_col_fire = image_collection.filterBounds(roi).filterDate(str(year) + '-05-1', str(year) + '-12-31').select('BurnDate')
    image_fire = image_col_fire.reduce(ee.Reducer.median())
    return image_fire

In [7]:
# tif to polygon
def tif_to_shp(image, year):
    if year != end_year and year != start_year:
        image = image.eq(year).Or(image.eq(year + 1)).Or(image.eq(year - 1))
    elif year != end_year:
        image = image.eq(year).Or(image.eq(year + 1))
    else:
        image = image.eq(year)
    image = image.updateMask(image.eq(1)).Or(image.eq(year - 1))
    return image.reduceToVectors(**{
        'scale': 30,
        'geometryType': "polygon", 
        'maxPixels': 1e13,
        'eightConnected': False
        }).select('count')


def mask_last_year(shp_pre, image, year, roi):
    image_mask = shp_pre.reduceToImage(['yod'], 'mean').Not().unmask(1).clip(roi)
#     Map.addLayer(image_mask, {}, 'image_' + str(year))
    return image.mask(image_mask)


def reduce_mask(shp, img_MASK):
    shp = feature_extract(shp, img_MASK, ['mask'])
    shp = sievel(shp, 'mask', 0.3, 'greater_than')
    return shp.select('count')


def ruduce_year(shp, img_CM, year):
    shp = feature_extract(shp, img_CM, ['yod'])
    shp = sievel(shp, 'yod', year + 0.7, 'less_than')
    return shp

In [8]:
# sievel
def sievel(shp, field, filter_count, filter_type):
    return shp.filterMetadata(field, filter_type, filter_count)


def bitwiseExtract(value, fromBit):
    maskSize = ee.Number(1).add(fromBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


def cloudfree_landsat(image):
    qa = image.select('pixel_qa')
    cloudState = bitwiseExtract(qa, 5) 
    cloudShadowState = bitwiseExtract(qa, 3)
    mask = cloudState.eq(0).And(cloudShadowState.eq(0))
    return image.updateMask(mask)

In [9]:
# area calculate
def area_calculate(shp):
    areaHa = shp.area(10)
    return shp.set({'area': areaHa})

In [10]:
def get_band_1(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B2')
    if year < 2013:
        image = image.select('B1')
    return image.clip(shp)

def get_band_2(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B3')
    if year < 2013:
        image = image.select('B2')
    return image.clip(shp)

def get_band_3(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B4')
    if year < 2013:
        image = image.select('B3')
    return image.clip(shp)

def get_band_4(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B5')
    if year < 2013:
        image = image.select('B4')
#         print(image.getInfo(), year, 'NDVI')
    return image.clip(shp)

def get_band_5(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = image.select('B6')
    if year < 2013:
        image = image.select('B5')
    return image.clip(shp)

def get_band_7(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    image = image.select('B6')
    return image.clip(shp)

'''feature extract'''

In [11]:
#ndvi nbr ndmi
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def addNDVI75(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)


def addNDMI(image):
    ndmi = image.normalizedDifference(['B5', 'B6']).rename('NDMI')
    return image.addBands(ndmi)


def addNDMI75(image):
    ndmi = image.normalizedDifference(['B4', 'B5']).rename('NDMI')
    return image.addBands(ndmi)


def addNBR(image):
    nbr = image.normalizedDifference(['B5', 'B7']).rename('NBR')
    return image.addBands(nbr)


def addNBR75(image):
    nbr = image.normalizedDifference(['B4', 'B7']).rename('NBR')
    return image.addBands(nbr)
    
 
def get_ndvi(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNDVI(image).select('NDVI')
#         print(image.getInfo(), year, 'NDVI')
    if year < 2013:
        image = addNDVI75(image).select('NDVI')
#         print(image.getInfo(), year, 'NDVI')
    return image

    
def get_ndmi(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNDMI(image).select('NDMI')
#         print(image.getInfo(), year, 'NDMI')
    if year < 2013:
        image = addNDMI75(image).select('NDMI')
#         print(image.getInfo(), year, 'NDMI')
    return image

    
    
def get_nbr(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').map(cloudfree_landsat).median()
    if year >= 2013:
        image = addNBR(image).select('NBR')
#         print(image.getInfo(), year, 'NBR')
    if year < 2013:
        image = addNBR75(image).select('NBR')
#         print(image.getInfo(), year, 'NBR')
    return image



def add_texture(nbr):
    square = ee.Kernel.square(**{'radius': 5})
    return nbr.multiply(1000).toInt().glcmTexture(**{
    'kernel': square,
    'size': 5,
    })


def feature_extract(shp, image, name):
    return image.reduceRegions(**{
    'collection': shp,
    'reducer': ee.Reducer.mean().setOutputs(name),
    'scale': 90
    })


# def get_index(ls, shp, year, org_year, time):
#     shp_feature = feature_extract(shp, get_ndvi(ls, shp, year), [time + '_NDVI_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_ndmi(ls, shp, year), [time + '_NDMI_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_nbr(ls, shp, year), [time + '_NBR_' + str(abs(year - org_year))])
#     return shp_feature

def get_index(ls, shp, year, org_year, time):
    shp_feature = feature_extract(shp, get_ndvi(ls, shp, year).select('NDVI'), [time + '_NDVI_' + str(abs(year - org_year))])
    shp_feature = feature_extract(shp_feature, get_ndmi(ls, shp, year).select('NDMI'), [time + '_NDMI_' + str(abs(year - org_year))])
    shp_feature = feature_extract(shp_feature, get_nbr(ls, shp, year).select('NBR'), [time + '_NBR_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_1(ls, shp, year), [time + '_B1_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_2(ls, shp, year), [time + '_B2_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_3(ls, shp, year), [time + '_B3_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_4(ls, shp, year), [time + '_B4_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_5(ls, shp, year), [time + '_B5_' + str(abs(year - org_year))])
#     shp_feature = feature_extract(shp_feature, get_band_7(ls, shp, year), [time + '_B7_' + str(abs(year - org_year))])
    return shp_feature

In [12]:
# process
def pre_interfere(shp, year0):
    if year0 - 2 <= start_year - 1:
        for year in range(start_year, year0):
            if year != 2012:
                shp = get_index(select_ls(year), shp, year, year0, 'PR')
    else:
        for year in range(year0 - 2, year0):
            if year != 2012:
                shp = get_index(select_ls(year), shp, year, year0, 'PR')
    return shp


def aft_interfere(shp, year0):
    if year0 + 5 >= end_year + 1:
        for year in range(year0, end_year + 1):
            if year != 2012:
                shp = get_index(select_ls(year), shp, year, year0, 'PT')
    else:
        for year in range(year0, year0 + 6):
            if year != 2012:
                shp = get_index(select_ls(year), shp, year, year0, 'PT')
    return shp


def select_ls(year):
    if year >=2013:
        ls = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    elif year >= 2003:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    elif year >= 1999:
#         ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").merge(ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA"))
        ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
    else:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    return ls


def process(shp, year):
    shp = sievel(shp, 'count', 10, 'greater_than')
    shp_feature = pre_interfere(shp, year)
    shp_feature = aft_interfere(shp_feature, year)
    if year != end_year:
        if year == 2011:
            texture = add_texture(get_nbr(select_ls(year + 2), shp, year + 2))
        else:
            texture = add_texture(get_nbr(select_ls(year + 1), shp, year + 1))
        shp_feature = feature_extract(shp_feature, texture.select('NBR_asm'), ['TXU_asm'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_idm'), ['TXU_HOM'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_contrast'), ['TXU_CON'])
        shp_feature = feature_extract(shp_feature, texture.select('NBR_sent'), ['TXU_ENT'])
#     print(shp_feature.first().getInfo(), '4')
    shp_feature = feature_extract(shp_feature, changeMap.select('mag'), ['LT_mag'])
    shp_feature = feature_extract(shp_feature, changeMap.select('dur'), ['LT_dur'])
    shp_feature = feature_extract(shp_feature, changeMap.select('dsnr'), ['LT_dsnr'])
    return shp_feature

In [13]:
#export
def export_to_driver(shp, year, x, y, filter_count):
    taskParams = {
    'driveFolder': 'NE_shp_30_6/1',
    'fileFormat': 'SHP'
    }
    task = ee.batch.Export.table(shp, str(year) + '_' + str(x) + '_' + str(y) + '_' + str(filter_count), taskParams)
    task.start()

In [14]:
def export_to_driver_4(shp, year, x, y, filter_count):
    taskParams = {
    'driveFolder': 'NE_shp_90_mask',
    'fileFormat': 'SHP'
    }
    task = ee.batch.Export.table(shp, str(year) + '_' + str(x) + '_' + str(y) + '_' + str(filter_count), taskParams)
    task.start()

In [15]:
def shp_part_process(shp_ogr, year, count_filter, shp_type, count, step):
    if shp_type == 'b':
        shp_p = process(sievel(shp_ogr, 'count', count_filter + step, 'greater_than'), year)
    else:
        shp_p = sievel(shp_ogr, 'count', count + step, 'not_greater_than')
        shp_p = sievel(shp_p, 'count', count, 'greater_than')
        shp_p = process(shp_p, year)
#         shp_p = process(sievel(shp_ogr, 'count', count_filter, 'not_greater_than'), year)
    return shp_p

In [16]:
# import os
# from tqdm import tqdm
# path = "D:\\data\\GEE\\NE_shp_30_mask\\"


# def get_path(year, i, j):
#     return [str(year) + '_' + str(i) + '_' + str(j + 1) + '_30.shp',
#            str(year) + '_' + str(i + 1) + '_' + str(j) + '_30.shp',
#            str(year) + '_' + str(i - 1) + '_' + str(j) + '_30.shp',
#            str(year) + '_' + str(i) + '_' + str(j - 1) + '_30.shp',]

In [17]:
# empty_path = []
# for year in tqdm(range(start_year, end_year + 1)):
#     year_path = path + str(year) + '\\'
#     exisit_path = os.listdir(path + str(year))
#     for i in range(115, 137):
#         for j in range(54, 37, -1):
#             count = 0
#             shp_path = year_path + str(year) + '_' + str(i) + '_' + str(j) +  '_30.shp'
#             if not os.path.exists(shp_path):
#                 for shp_name in get_path(year, i, j):
#                     other_shp_path = year_path + shp_name
#                     if os.path.exists(other_shp_path):
#                         count += 1
#                 if count >= 2:
#                     empty_path.append([year, i, j])

In [18]:
# for year, i, j in tqdm(empty_path):
# #     roi_list = [(i, j), (i + 0.5, j), (i, j - 0.5), (i + 0.5, j - 0.5)]
# #     count = 1
# #     for m, n in roi_list:
# #         roi = ee.Geometry.Polygon(
# #             [[m-0.01,n+0.01],[m+0.51,n+0.01],[m+0.51,n-0.51],[m-0.01,n-0.51]])
#         roi = ee.Geometry.Polygon(
#              [[i-0.01,j+0.01],[i+1.01,j+0.01],[i+1.01,j-1.01],[i-0.01,j-1.01]])
#         exportImg = changeMap.clip(roi).short().select('yod')
#         exportImg = exportImg.unmask(0).focal_median(**{
#         'radius': 3.5,
#         'kernelType': 'square'
#         }).toInt()
#         maskImg = IFZ_mask.clip(roi)
#         if year != 2012:
#             shp_ogr = tif_to_shp(exportImg, year)
#             shp_ogr = reduce_mask(shp_ogr, maskImg)
#             if year > 2000:
#                 image_fire = fire_process(roi, image_collection_fire, year).unmask(0)
#                 shp_fire_classify = feature_extract(shp_ogr, image_fire, ['fire_class'])
#                 shp_unfire = sievel(shp_fire_classify, 'fire_class', 0, 'equals')
#                 shp_fire = sievel(shp_fire_classify, 'fire_class', 0, 'greater_than')
#                 shp = process(shp_ogr, year).merge(shp_fire)
#             else:
#                 shp = process(shp_ogr, year)
#         export_to_driver_4(shp, year, i, j, 90)

In [19]:
# def calculation_IFZ(FZ):
#     IFZ = FZ.expression(
#     '(B3 ** 2 + B5 ** 2 + B7 ** 2) / 3',
#     {
#         'B3': FZ.select('B3'),
#         'B5': FZ.select('B5'),
#         'B7': FZ.select('B7')
#     }).sqrt()
#     return IFZ


# def calculation_FZ(image_cal, image_target, band):
#     mean = image_cal.reduceRegions(**{
#     'collection': protect_area,
#     'reducer': ee.Reducer.mean(),
# #         .setOutputs(name),
#     'scale': 30
#     }).aggregate_mean(band)
#     std = image_cal.reduceRegions(**{
#     'collection': protect_area,
#     'reducer': ee.Reducer.stdDev(),
# #         .setOutputs(name),
#     'scale': 30
#     }).aggregate_mean(band)
#     FZ = image_target.expression(
#     '(B - B_mean) / B_std',
#     {
#         'B': image_target.select(band),
#         'B_mean': mean,
#         'B_std': std
#     })
#     return FZ



# def select_ls357(year):
#     if year >=2013:
#         ls = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").select(['B4', 'B6', 'B7', 'pixel_qa'])
#     elif year >= 2003:
#         ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").select(['B3', 'B5', 'B7', 'pixel_qa'])
#     elif year >= 1999:
#         ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").select(['B3', 'B5', 'B7', 'pixel_qa'])
#     else:
#         ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").select(['B3', 'B5', 'B7', 'pixel_qa'])
#     return ls


# def get_IFZ(year):
#     ls = select_ls357(year)
#     image_pa = ls.filterBounds(protect_area).filterDate(str(year) + '-06-1',str(year) + '-9-30').map(cloudfree_landsat).median().rename(['B3', 'B5', 'B7', 'pixel_qa'])
#     mask_image = ls.filterBounds(roi).filterDate(str(year) + '-06-1',str(year) + '-9-30').map(cloudfree_landsat).median().rename(['B3', 'B5', 'B7', 'pixel_qa'])
#     FZ = calculation_FZ(image_pa, mask_image, 'B3').addBands(calculation_FZ(image_pa, mask_image, 'B5')).addBands(calculation_FZ(image_pa, mask_image, 'B7'))
#     IFZ = ee.ImageCollection(calculation_IFZ(FZ).lte(3))
#     return IFZ

# # def forest_mask(changeMap):
# IFZ_list = []
# roi = ee.Geometry.Polygon([[114, 55], [137, 55], [137, 37], [114, 37]])
# aoi = ee.Geometry.Polygon([[123, 52], [126, 52], [126, 50], [123, 50]])
# windows_size = 5
# for syear in range(start_year, end_year - windows_size + 1):
#     if syear <= 1989:
#         IFZ_pre = get_IFZ(syear)
#         if syear == start_year:
#             IFZ_col_pre = IFZ_pre   
#         else:
#             IFZ_col_pre = IFZ_col_pre.merge(IFZ_pre)
#     for year in range(syear, syear + windows_size):
#         if year != 2012:
#             IFZ = get_IFZ(year)
#         else:
#             IFZ = get_IFZ(year - 1)
#         if year == syear:
#             IFZ_col = IFZ
#         else:
#             IFZ_col = IFZ_col.merge(IFZ)
#     IFZ_all = ee.ImageCollection(IFZ_col.sum().eq(windows_size))
#     if syear == start_year:
#         IFZ_all_all = IFZ_all
#     else:
#         IFZ_all_all = IFZ_all_all.merge(IFZ_all)
# IFZ_mask_win = IFZ_all_all.sum().gt(0)
# IFZ_mask_pre = IFZ_col_pre.sum().gt(0)
# IFZ_mask = IFZ_mask_win.add(IFZ_mask_pre).gt(0)
# # forest_mask(changeMap)

In [20]:
# task = ee.batch.Export.image.toAsset(image=IFZ_mask,
# assetId="users/dxj/changeMap_mask",
# scale=30,
# maxPixels=1e13,
# region=roi
# )
# #任务启动
# task.start()

In [21]:
# Map = geemap.Map(center=(44, 125), zoom=5)
# Map

In [22]:
# start_year = 2002
# end_year = 2004
# [(117, 52),(117, 54), (125, 53), (116, 54), (116, 52), (115, 53),
#              (115, 54), (117, 53), (124, 54), (118, 53), (119, 53), (120, 53)]:
# [(122,52)]:

# for i, j in [(122, 52), (123, 52), (128, 48), (129, 48)]:
for i in range(116, 117):
    for j in range(42, 54):
        roi = ee.Geometry.Polygon(
             [[i-0.01,j+0.01],[i+1.01,j+0.01],[i+1.01,j-1.01],[i-0.01,j-1.01]])
        exportImg = changeMap.clip(roi).short().select('yod')
        exportImg = exportImg.unmask(0).focal_median(**{
        'radius': 2.5,
        'kernelType': 'square'
        }).toInt()
        maskImg = IFZ_mask.clip(roi)
        exportImg = exportImg.mask(exportImg.eq(0).Not())
        for year in range(start_year, end_year + 1):
            if year != 2012:
                if year != start_year:
                    shp_ogr = tif_to_shp(mask_last_year(shp_pre, exportImg, year, roi), year)
                    shp_pre = ruduce_year(shp_ogr, exportImg, year)
                    shp_ogr = reduce_mask(shp_pre, maskImg)
                else:
                    shp_ogr = tif_to_shp(exportImg, year)
                    shp_pre = ruduce_year(shp_ogr, exportImg, year)
                    shp_ogr = reduce_mask(shp_pre, maskImg)
                if year > 2000:
                    image_fire = fire_process(roi, image_collection_fire, year).unmask(0)
                    shp_fire_classify = feature_extract(shp_ogr, image_fire, ['fire_class'])
                    shp_unfire = sievel(shp_fire_classify, 'fire_class', 0, 'equals')
                    shp_unfire = shp_unfire.set('fire_class', 1)
                    shp_fire = sievel(shp_fire_classify, 'fire_class', 0, 'greater_than')
                    shp_fire = shp_fire.set('fire_class', 0)
                    shp = process(shp_unfire, year).merge(shp_fire)
                else:
                    shp = process(shp_ogr, year)
                export_to_driver(shp, year, i, j, 30)
#         Map.addLayer(shp_ogr, {}, 'shp_' + str(year))
#     Map.addLayer(exportImg, {'min': 1986, 'max': 2019}, 'changeMap')

In [23]:
# # step = 500
# # count_filter = 5000
# for i in range(131, 132):
#     for j in range(49, 37, -1):
# # for i in range(130, 131):
# #     for j in range(44, 37, -1):
# # # for i, j in [(125, 53),(117, 54), (117, 52), (116, 54), (116, 52), (115, 53),
# # #              (115, 54), (117, 53), (124, 54), (118, 53), (119, 53), (120, 53)]:
for i in range(117, 137):
    for j in range(38, 54):
        roi = ee.Geometry.Polygon(
             [[i-0.01,j+0.01],[i+1.01,j+0.01],[i+1.01,j-1.01],[i-0.01,j-1.01]])
        exportImg = changeMap.clip(roi).short().select('yod')
        exportImg = exportImg.unmask(0).focal_median(**{
        'radius': 2.5,
        'kernelType': 'square'
        }).toInt()
        maskImg = IFZ_mask.clip(roi)
        exportImg = exportImg.mask(exportImg.eq(0).Not())
        for year in range(start_year, end_year + 1):
            if year != 2012:
                if year != start_year:
                    shp_ogr = tif_to_shp(mask_last_year(shp_pre, exportImg, year, roi), year)
                    shp_pre = ruduce_year(shp_ogr, exportImg, year)
                    shp_ogr = reduce_mask(shp_pre, maskImg)
                else:
                    shp_ogr = tif_to_shp(exportImg, year)
                    shp_pre = ruduce_year(shp_ogr, exportImg, year)
                    shp_ogr = reduce_mask(shp_pre, maskImg)
                if year > 2000:
                    image_fire = fire_process(roi, image_collection_fire, year).unmask(0)
                    shp_fire_classify = feature_extract(shp_ogr, image_fire, ['fire_class'])
                    shp_unfire = sievel(shp_fire_classify, 'fire_class', 0, 'equals')
                    shp_unfire = shp_unfire.set('fire_class', 1)
                    shp_fire = sievel(shp_fire_classify, 'fire_class', 0, 'greater_than')
                    shp_fire = shp_fire.set('fire_class', 0)
                    shp = process(shp_unfire, year).merge(shp_fire)
                else:
                    shp = process(shp_ogr, year)
                export_to_driver(shp, year, i, j, 30)

KeyboardInterrupt: 

In [ ]:
# # step = 500
# # count_filter = 5000
# for i in range(132, 137):
#     for j in range(54, 37, -1):
# # for i in range(130, 131):
# #     for j in range(44, 37, -1):
# # # for i, j in [(125, 53),(117, 54), (117, 52), (116, 54), (116, 52), (115, 53),
# # #              (115, 54), (117, 53), (124, 54), (118, 53), (119, 53), (120, 53)]:
#         for year in range(start_year, end_year + 1):
# #         if year == 2003:
#             roi = ee.Geometry.Polygon(
#                 [[i-0.01,j+0.01],[i+1.01,j+0.01],[i+1.01,j-1.01],[i-0.01,j-1.01]])
#             exportImg = changeMap.clip(roi).short()
#             shp_ogr = tif_to_shp(exportImg.select('yod'), year)
#             image_fire = fire_process(roi, image_collection_fire, year).unmask(0)
#             shp_fire_classify = feature_extract(shp_ogr, image_fire, ['fire_class'])
#             shp_unfire = sievel(shp_fire_classify, 'fire_class', 0, 'equals')
#             shp_unfire = shp_unfire.set('fire_class', 1)
#             shp_fire = sievel(shp_fire_classify, 'fire_class', 0, 'greater_than')
#             shp_fire = shp_fire.set('fire_class', 0)
# #                 shp_s = shp_part_process(shp_unfire, year, count_filter, 's', 5, step)
# #                 for count in tqdm(range(step + 5, count_filter + 6, step), desc=str(i) + '_' + str(j)):
# #                     shp_s = shp_s.merge(shp_part_process(shp_unfire, year, count_filter, 's', count, step))
# #                 shp_b = shp_part_process(shp_unfire, year, count_filter, 'b', count, step)
# #                 shp = shp_s.merge(shp_b).merge(shp_fire)
# #                 print(ee.Algorithms.If(shp.first(), 1, 0).getInfo())
#             shp = process(sievel(shp_unfire, 'count', 5, 'greater_than'), year).merge(shp_fire)
# #                 shp = process(shp_ogr, year)
#             export_to_driver(shp, year, i, j, 30)

In [ ]:
# import os
# os.system('shutdown /s /t 0’)